In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

import cv2
import matplotlib.pyplot as plt

from itertools import combinations

In [2]:
a = pd.read_csv('../output/augmentation10_pred_256_14.csv')
meta = pd.read_csv('../datasets_fin/meta.csv')

In [3]:
a[a['target']==0]

,ID,target
6,00b33e0ee6d59427.jpg,0
25,01ebd05a14e10618.jpg,0
28,0298c2151b43d86b.jpg,0
31,02b3712bd48b8644.jpg,0
44,048efa3d18a4999d.jpg,0
...,...,...
3077,fb0cae80fe2ac9fc.jpg,0
3089,fc23b6bc90ba1e39.jpg,0
3091,fc2c5f6e5d40ce18.jpg,0
3111,fd5307c7a3700629.jpg,0


In [4]:
# idx로 img 가져오기
def get_item(idx):
    name, target = a.loc[idx,]
    img = np.array(Image.open(os.path.join("../datasets_fin/test/", name)))

    # img = trn_transform(image=img)['image']
    return img, target, name

In [5]:
meta['class_name_kr'] = ['계좌번호','임신 의료비 지급 신청서','차량 계기판','입퇴원 확인서','진단서','운전면허','진료비 영수증','진료 확인서',
                         '주민등록증','여권','진료비 납입 확인서','약 영수증','처방전','이력서','(진료)소견서','차량 등록증','차량 번호판']

In [6]:
meta

,target,class_name,class_name_kr
0,0,account_number,계좌번호
1,1,application_for_payment_of_pregnancy_medical_e...,임신 의료비 지급 신청서
2,2,car_dashboard,차량 계기판
3,3,confirmation_of_admission_and_discharge,입퇴원 확인서
4,4,diagnosis,진단서
5,5,driver_lisence,운전면허
6,6,medical_bill_receipts,진료비 영수증
7,7,medical_outpatient_certificate,진료 확인서
8,8,national_id_card,주민등록증
9,9,passport,여권


In [7]:
a.value_counts('target').sort_index()

target
0     218
1      88
2     199
3     140
4     142
5     200
6     234
7     116
8     199
9     199
10    255
11    184
12    197
13    164
14    174
15    231
16    200
Name: count, dtype: int64

In [1]:
# # target 값에 따른 img 확인
# for idx in a[a['target'] == 0].index:
#     img, target, name = get_item(idx)
#     print(img.shape, name)
#     plt.imshow(img)
#     plt.show()
#     # print(idx)